In [ ]:
import os
import sys

sys.path.append('../..')
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import matplotlib.pyplot as plt
import mdtraj as md
import numpy as np
import tensorflow as tf

from ase import Atoms
from ase.build import make_supercell
from ase.optimize import BFGS, LBFGS
from ase.vibrations import Vibrations
from ase.md import MDLogger
from ase.md.velocitydistribution import (MaxwellBoltzmannDistribution,Stationary, ZeroRotation)
from ase.md.langevin import Langevin
from ase.md.andersen import Andersen
from ase.units import fs
from ase.io import read
from ase.io.trajectory import Trajectory
from Barostat import MonteCarloBarostat

from HybridFF import HybridFF
from Utilities import show_results, write_xyz, validate
from Constants import EV_TO_KJ, H_TO_KJ, BOHR_TO_ANGSTROM


In [ ]:
SYSTEM = 'XXVI'

DATA = np.load('../../../data/test_sets/DATA_CSP_ranking.npy', allow_pickle=True).item()
MIN_STRUCTURES = np.load(f'data/STRUCTURES_{SYSTEM}.npy', allow_pickle=True).item()
MIN_CELLS =  np.load(f'data/MIN_CELLS_{SYSTEM}.npy', allow_pickle=True).item()
MD_DATA = np.load(f'data/MD_TERMS_{SYSTEM}.npy', allow_pickle=True).item()
barostat = MonteCarloBarostat()

In [ ]:
THERMAL_CELLS = {}
for POLY_KEY in MD_DATA:
    mol_size = len(DATA[SYSTEM][POLY_KEY]['monomer_symbols'])
    elements_uc = DATA[SYSTEM][POLY_KEY]['uc_symbols']
    coords_uc = MIN_STRUCTURES[POLY_KEY]['coordinates']
    cell = MIN_STRUCTURES[POLY_KEY]['cell'][0]
    cell_md = MD_DATA[POLY_KEY]['cell']
    n_molecules = len(elements_uc) // mol_size
    length_scale = np.diag((cell_md / cell)).reshape(1, 1, 3)
    coords_scaled = barostat._scale_coordinates(coords_uc, length_scale, n_molecules)
    THERMAL_CELLS[POLY_KEY] = {}
    THERMAL_CELLS[POLY_KEY]['coords'] = coords_scaled
    THERMAL_CELLS[POLY_KEY]['cells'] = cell_md[None]

In [ ]:
np.save(f'data/THERMAL_CELLS_{SYSTEM}', THERMAL_CELLS)